<div style="display: flex; justify-content: space-between; align-items: center;">
    <div style="text-align: left; flex: 4;">
        <strong>Author:</strong> Amirhossein Heydari — 
        📧 <a href="mailto:amirhosseinheydari78@gmail.com">amirhosseinheydari78@gmail.com</a> — 
        🐙 <a href="https://github.com/mr-pylin/media-processing-workshop" target="_blank" rel="noopener">github.com/mr-pylin</a>
    </div>
    <div style="display: flex; justify-content: flex-end; flex: 1; gap: 8px; align-items: center; padding: 0;">
        <a href="https://opencv.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/opencv/logo/OpenCV_logo_no_text-1.svg"
                 alt="OpenCV Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://pillow.readthedocs.io/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/pillow/logo/pillow-logo-248x250.png"
                 alt="PIL Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scikit-image.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scikit-image/logo/logo.png"
                 alt="scikit-image Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scipy.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scipy/logo/logo.svg"
                 alt="SciPy Logo"
                 style="max-height: 48px; width: auto;">
        </a>
    </div>
</div>
<hr>


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Transforms](#toc2_)    
  - [Fourier](#toc2_1_)    
    - [Fourier Series (CTFS & DTFS)](#toc2_1_1_)    
    - [Fourier Transform (CTFT & DTFT)](#toc2_1_2_)    
    - [Discrete Fourier Transform (DFT)](#toc2_1_3_)    
    - [Fast (Discrete) Fourier Transform (FFT)](#toc2_1_4_)    
    - [Short-Time Fourier Transform (STFT)](#toc2_1_5_)    
    - [Gibbs Phenomenon](#toc2_1_6_)    
    - [Examples](#toc2_1_7_)    
      - [Example 1: Dirac Delta Function (Discrete)](#toc2_1_7_1_)    
      - [Example 2: Rectangular (Box) Pulse](#toc2_1_7_2_)    
      - [Example 3: Gaussian Signal](#toc2_1_7_3_)    
  - [Cosine Transform](#toc2_2_)    
    - [Example](#toc2_2_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from numpy.typing import NDArray

In [ ]:
# disable automatic figure display (plt.show() required)  
# this ensures consistency with .py scripts and gives full control over when plots appear
plt.ioff()

In [ ]:
# set the default marker size for all plots
plt.rcParams["lines.markersize"] = 5

# <a id='toc2_'></a>[Transforms](#toc0_)

- Transforms are operations that **change the representation** of data from one domain to another.
- They are used to **simplify problems**, **reveal hidden properties**, or enable operations that are **easier in a different domain**.

🔢 **Mathematical Formulas:**

- Forward Transform
$$T\{ f(x, y) \} = F(u, v)$$

- Backward Transform
$$T^{-1}\{ F(u, v) \} = f(x, y)$$

**Equality of Original and Reconstructed Signal:**

- If the transform is **lossless** e.g., **Fourier Transform** and **Wavelet Transform**, the backward transform **perfectly** reconstructs the original signal.
- In the **discrete world**, however, the situation is different due to **practical limitations**:
  - Finite Precision Arithmetic:
    - Computers use a finite number of bits to represent numbers (e.g., 32-bit or 64-bit floating-point).
    - This introduces rounding errors during calculations.
  - Discretization:
    - Signals are sampled and quantized, which means they are represented by a finite set of values.
    - This can lead to loss of information.

$$T^{-1}\{ T\{ f(x, y) \} \} \approx f(x, y)$$

**Transform $\mathbf{T}$:**

- A transform $T$ can be defined as an **integral operator** (for continuous functions) or a **summation** (for discrete functions).
- $K$ is known as the **kernel of the transform** and defines the specific operation.
  - **1D Transform**:
    $$T\{f(t)\} = \int_{-\infty}^{\infty} f(t) \, K(t, \omega) \, dt$$
    $$T\{f[n]\} = \sum_{n=-\infty}^{\infty} f[n] \, K[n,\omega]$$

  - **2D Transform**:
    $$T\{f(x,y)\} = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} f(x,y) \, K(x,y;\omega_x,\omega_y) \, dx \, dy$$
    $$T\{f[m,n]\} = \sum_{m=-\infty}^{\infty}\sum_{n=-\infty}^{\infty} f[m,n] \, K[m,n;\omega_x,\omega_y]$$

✍️ **Notes:**

- **Convolution** is not typically classified as a **transform**.
- It is an operation that **combines two functions** to produce a **third function**.


## <a id='toc2_1_'></a>[Fourier](#toc0_)


### <a id='toc2_1_1_'></a>[Fourier Series (CTFS & DTFS)](#toc0_)

- It decomposes periodic functions into a sum of sine and cosine waves (or complex exponentials) of different frequencies.
- It was developed by Joseph Fourier in the early 19th century to solve heat transfer problems.

**Key Idea:**
- Any periodic function $f(t)$ with period $T$ can be expressed as:
  $$f(t) = a_0 + \sum_{n=1}^{\infty} \left[ a_n \cos(n\omega_0 t) + b_n \sin(n\omega_0 t) \right]$$
- where
  - $\omega_0 = \frac{2 \pi}{T}$ is the **fundamental frequency**,
  - $a_0$, $a_n$, and $b_n$ are** Fourier coefficients** representing the **contribution of each frequency component**.

**Deriving the Fourier Coefficients:**
- The coefficients $a_0$, $a_n$, and $b_n$ are calculated using **orthogonality** of **sine** and **cosine** functions over one period $T$.
  $$a_0 = \frac{1}{T} \int_{T} f(t) \, dt, \qquad a_n = \frac{2}{T} \int_{T} f(t) \cos(n\omega_0 t) \, dt, \qquad b_n = \frac{2}{T} \int_{T} f(t) \sin(n\omega_0 t) \, dt$$

**Exponential Form of Fourier Series:**
- Using Euler’s formula $e^{i \theta} = cos(\theta) + i*sin(\theta)$, we can rewrite the Fourier Series in a more compact **complex exponential form**:
  $$f(t) = \sum_{n=-\infty}^{\infty} c_n e^{in\omega_0 t}$$
- where the complex coefficients $c_n$ are:
  $$c_n = \frac{1}{T} \int_{T} f(t) e^{-in\omega_0 t} \, dt.$$
- The relationship between $c_n$ and $a_n$, $b_n$ is:
  $$c_n = \frac{a_n - ib_n}{2}, \quad c_{-n} = \frac{a_n + ib_n}{2}, \quad c_0 = a_0.$$

**Periodicity in Time and Frequency Domains:**

| Signal Type         | Time Domain Periodicity    | Frequency Domain Representation                         |
|---------------------|----------------------------|---------------------------------------------------------|
| **Continuous-Time** | Periodic with period $T$     | Discrete coefficients {$c_k$} (non-periodic in $k$)         |
| **Discrete-Time**   | Periodic with period $N$     | Discrete coefficients {$a[k]$} (periodic with period $N$)   |

📝 **Paper**:

- [**Théorie analytique de la chaleur**](https://www.sciencedirect.com/science/article/abs/pii/B9780444508713501078) by [Joseph Fourier](https://en.wikipedia.org/wiki/Joseph_Fourier) in *1822*.


### <a id='toc2_1_2_'></a>[Fourier Transform (CTFT & DTFT)](#toc0_)

- The Fourier Series works for periodic signals (repeating patterns).
- The Fourier Transform generalizes the Fourier Series for non-periodic signals

**Transition to the Fourier Transform:**
- Treat a non-periodic signal as a periodic signal with an infinite period ($T \to \infty$)
  - The function no longer repeats—it becomes aperiodic.
  - The fundamental frequency $\omega_0 = \frac{2 \pi}{T}$ gets **smaller** as $T$ increases.
  - The **discrete** frequency components in the **Fourier Series** become **infinitely close** together, forming a **continuous** spectrum.
  - Instead of a sum of discrete harmonics, we get an integral over all frequencies, which is exactly the Fourier Transform.
  
  $$\lim_{T \to \infty} \text{Fourier Series} = \text{Fourier Transform}$$

**Forward Fourier Transform:**
$$F(\omega) = \int_{-\infty}^{\infty} f(t) e^{-i\omega t} dt$$

**Inverse Fourier Transform:**
$$f(t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} F(\omega) e^{i\omega t} d\omega$$

**Periodicity in Time and Frequency Domains:**

| Signal Type         | Time Domain                | Frequency Domain Representation                                   |
|---------------------|----------------------------|-------------------------------------------------------------------|
| **Continuous-Time** | Non-periodic signal $x(t)$   | Continuous function $X(j \omega)$ (non-periodic, $ω \in \mathbb{R}$)                     |
| **Discrete-Time**   | Discrete signal $x[n]$       | Continuous function $X(e^{j \omega})$ (periodic with period $2 \pi$, typically defined over $[–\pi, \pi]$) |

📝 **Paper**:

- [**Théorie analytique de la chaleur**](https://www.sciencedirect.com/science/article/abs/pii/B9780444508713501078) by [Joseph Fourier](https://en.wikipedia.org/wiki/Joseph_Fourier) in *1822*.


### <a id='toc2_1_3_'></a>[Discrete Fourier Transform (DFT)](#toc0_)

- It is the digital version of the Fourier Transform, used when dealing with discrete data or signals.

**Forward Discrete Fourier Transform:**
$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \frac{kn}{N}}  \quad \text{for} \quad k = 0, 1, \dots, N-1$$

**Inverse Discrete Fourier Transform:**
$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] e^{i 2 \pi \frac{kn}{N}}  \quad \text{for} \quad n = 0, 1, \dots, N-1$$

**Periodicity in Time and Frequency Domains:**

| Signal Type | Time Domain Signal                                               | Frequency Domain Signal                                     |
|:-----------:|------------------------------------------------------------------|-------------------------------------------------------------|
| **DFT**     | Finite-length sequence $x[n]$ (assumed periodic with period $N$) | Finite-length sequence $X[k]$ (periodic with period $N$)    |

📝 **Book**:

- [**Theoria Interpolationis Methodo Nova Tractata**](https://gdz.sub.uni-goettingen.de/id/DE-611-HS-3373323) by [Carl Friedrich Gauss](https://gdz.sub.uni-goettingen.de/suche?filter%5B0%5D%5Bfacet_creator_personal%5D=Gau%C3%9F,%20Carl%20Friedrich).


### <a id='toc2_1_4_'></a>[Fast (Discrete) Fourier Transform (FFT)](#toc0_)

- The FFT is an algorithm to compute Discrete Fourier Transform (DFT) efficiently ($O(NlogN)$).

🪜 **Step-by-Step Explanation:**

1. **Input Sequence:**  
   Start with a sequence $x[n]$ for $n = 0, 1, \dots, N-1$ (typically, $N$ is a power of $2$).

2. **Divide into Even and Odd Parts:**  
   Split the sequence into two subsequences:  
   - Even-indexed:  
     $$x_{even}[m] = x[2m], \quad m = 0, 1, \dots, \frac{N}{2 - 1}$$
   - Odd-indexed:  
     $$x_{odd}[m] = x[2m+1], \quad m = 0, 1, \dots, \frac{N}{2 - 1}$$

3. **Recursively Compute DFTs:**  
   Compute the DFTs of the even and odd parts separately:  
   $$E[k] = DFT(x_{even}[m]), \qquad O[k] = DFT(x_{odd}[m]), \qquad \text{for } k = 0, 1, \dots, N/2 - 1.$$

4. **Combine Using Butterfly Operations:**  
   Use the symmetry of complex exponentials (the twiddle factors) to merge the two smaller DFTs into one DFT of length $N$. For $k = 0, 1, \dots, \frac{N}{2 - 1}$, compute:  
   $$X[k] = E[k] + W_N^k * O[k]$$
   $$X[k + \frac{N}{2}] = E[k] - W_N^k * O[k]$$
   where the twiddle factor is given by:  
   $$W_N^k = e^{-j 2πk/N}$$

5. **Output the Combined DFT:**  
   The sequence $X[k]$ for $k = 0, 1, \dots, N-1$ is the DFT of the original sequence $x[n]$.

📝 **Papers**:

- [**An Algorithm for the Machine Calculation of Complex Fourier Series**](https://doi.org/10.1090/S0025-5718-1965-0178586-1) by [James W. Cooley](https://en.wikipedia.org/wiki/James_Cooley) and [John W. Tukey](https://en.wikipedia.org/wiki/John_Tukey) in *1965*.
- [**Gauss and the History of the Fast Fourier Transform**](https://link.springer.com/article/10.1007/BF00348431) by [Michael T. Heideman](https://link.springer.com/search?sortBy=newestFirst&dc.creator=Michael%20T.%20Heideman) et al. in *1985*.


### <a id='toc2_1_5_'></a>[Short-Time Fourier Transform (STFT)](#toc0_)

- It is used to analyze signals whose frequency content changes over time.
- It combines aspects of both the Fourier Transform and time-domain analysis by performing a Fourier transform on a sliding window over the signal.
- the STFT reveals local frequency information within a specific window of time.

$$\text{STFT}(x(t), \tau, \omega) = \int_{-\infty}^{\infty} x(t) w(t - \tau) e^{-j \omega t} \, dt$$


### <a id='toc2_1_6_'></a>[Gibbs Phenomenon](#toc0_)

- When **approximating** a piecewise continuously differentiable function with a jump discontinuity using its Fourier series.
- The partial sums exhibit **overshoots** and **undershoots** near the **discontinuity**.

**Overshoot Amplitude:**
- As $N \to \infty$, the maximum overshoot approaches
  $$\lim_{N \to \infty} \max \left( S_N(t) \right) \approx 1.0895$$

In [ ]:
def square_wave(t: NDArray, T: float = 2 * np.pi) -> NDArray:
    return np.where(np.mod(t, T) < T / 2, 1, -1)


# fourier series approximation for square wave
def fourier_series(t: NDArray, N: int, T: float = 2 * np.pi) -> NDArray:
    a_0 = 0  # for a square wave, the DC component is 0
    result = a_0
    for n in range(1, N + 1, 2):  # sum only odd harmonics (1, 3, 5, ...)
        result += (4 / (np.pi * n)) * np.sin(n * t)  # fourier series terms for square wave
    return result


# time values
t = np.linspace(0, 2 * np.pi, 1000)

# original square wave (exact signal)
original_square_wave = square_wave(t)

# fourier series approximations with different numbers of terms
N_values = [3, 5, 10, 50]

# plot
plt.figure(figsize=(16, 7))
plt.plot(t, original_square_wave, label="Original Square Wave", color="white", linewidth=2)
for N in N_values:
    approx_wave = fourier_series(t, N)
    plt.plot(t, approx_wave, label=f"Fourier Series with {N} terms")
plt.title("Gibbs Phenomenon - Fourier Series Approximation of Square Wave")
plt.xlabel("Time (t)")
plt.ylabel("Amplitude")
plt.legend(fontsize=12)
plt.grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

### <a id='toc2_1_7_'></a>[Examples](#toc0_)


#### <a id='toc2_1_7_1_'></a>[Example 1: Dirac Delta Function (Discrete)](#toc0_)


In [ ]:
# parameters
fs = 11  # length of the signal
t = np.arange(fs)  # discrete time indices
freqs = np.fft.fftshift(np.fft.fftfreq(fs, d=1 / fs))

# dirac delta function (impulse at center)
x = np.zeros(fs)
x[0] = 1
X = np.fft.fftshift(np.fft.fft(x))

# plot
fig, axes = plt.subplots(1, 4, figsize=(18, 4), layout="compressed")
axes[0].stem(t, x, basefmt=" ", linefmt="b", markerfmt="bo")
axes[0].set_title("Time Domain")
axes[0].set_xlabel("Sample Index")
axes[0].set_ylabel("Amplitude")
axes[0].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[0].set_xlim([-1, fs])
axes[1].stem(freqs, X.real, basefmt=" ", linefmt="r", markerfmt="ro")
axes[1].set_title("FFT - Real Part")
axes[1].set_xlabel("Frequency Index")
axes[1].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[1].set_xlim([-(fs // 2) - 1, fs // 2 + 1])
axes[2].stem(freqs, X.imag, basefmt=" ", linefmt="g", markerfmt="go")
axes[2].set_title("FFT - Imaginary Part")
axes[2].set_xlabel("Frequency Index")
axes[2].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[2].set_xlim([-(fs // 2) - 1, fs // 2 + 1])
axes[3].stem(freqs, np.abs(X), basefmt=" ", linefmt="m", markerfmt="mo")
axes[3].set_title("FFT - Magnitude")
axes[3].set_xlabel("Frequency Index")
axes[3].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[3].set_xlim([-(fs // 2) - 1, fs // 2 + 1])
plt.show()

#### <a id='toc2_1_7_2_'></a>[Example 2: Rectangular (Box) Pulse](#toc0_)


In [ ]:
# parameters
L = 100  # total signal length
N = 10  # width of the square pulse (1 from 0 to 10)
fs = L  # sampling rate (assumed 1 sample per unit interval)
t = np.arange(L)  # discrete time indices
freqs = np.fft.fftshift(np.fft.fftfreq(L, d=1 / fs))

# define the box function
x = np.zeros(L)
x[:N] = 1

# compute FFT and shift
X = np.fft.fftshift(np.fft.fft(x))

# plot
fig, axes = plt.subplots(1, 4, figsize=(18, 4), layout="compressed")
axes[0].stem(t, x, basefmt=" ", linefmt="b", markerfmt="bo")
axes[0].set_title("Time Domain")
axes[0].set_xlabel("Sample Index")
axes[0].set_ylabel("Amplitude")
axes[0].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[0].set_xlim([-1, L])
axes[1].stem(freqs, X.real, basefmt=" ", linefmt="r", markerfmt="ro")
axes[1].set_title("FFT - Real Part")
axes[1].set_xlabel("Frequency Index")
axes[1].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[1].set_xlim([-(L // 2) - 1, L // 2 + 1])
axes[2].stem(freqs, X.imag, basefmt=" ", linefmt="g", markerfmt="go")
axes[2].set_title("FFT - Imaginary Part")
axes[2].set_xlabel("Frequency Index")
axes[2].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[2].set_xlim([-(L // 2) - 1, L // 2 + 1])
axes[3].stem(freqs, np.abs(X), basefmt=" ", linefmt="m", markerfmt="mo")
axes[3].set_title("FFT - Magnitude")
axes[3].set_xlabel("Frequency Index")
axes[3].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[3].set_xlim([-(L // 2) - 1, L // 2 + 1])
plt.show()

#### <a id='toc2_1_7_3_'></a>[Example 3: Gaussian Signal](#toc0_)


In [ ]:
# parameters
L = 512  # total signal length
mu = L // 2  # center of gaussian
sigma = 40

t = np.arange(L)  # time indices
freqs = np.fft.fftshift(np.fft.fftfreq(L, d=1 / L))

# create Gaussian signal
x = np.exp(-((t - mu) ** 2) / (2 * sigma**2))

# compute FFT
X = np.fft.fftshift(np.fft.fft(x))
X_magnitude = np.abs(X) / np.max(np.abs(X))  # Normalize for better visualization

# downsample indices for the time-domain stem plot
step = 8
t_down = t[::step]
x_down = x[::step]

# plot
fig, axes = plt.subplots(1, 4, figsize=(18, 4), layout="compressed")
axes[0].stem(t_down, x_down, basefmt=" ", linefmt="b", markerfmt="bo")
axes[0].set_title("Time Domain (Gaussian Signal)")
axes[0].set_xlabel("Sample Index")
axes[0].set_ylabel("Amplitude")
axes[0].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[0].set_xlim([-1, L])
axes[1].stem(freqs, X.real, basefmt=" ", linefmt="r", markerfmt="ro")
axes[1].set_title("FFT - Real Part [zoomed in]")
axes[1].set_xlabel("Frequency Index")
axes[1].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[1].set_xlim([-L // 10, L // 10])
axes[2].stem(freqs, X.imag, basefmt=" ", linefmt="g", markerfmt="go")
axes[2].set_title("FFT - Imaginary Part [zoomed in]")
axes[2].set_xlabel("Frequency Index")
axes[2].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[2].set_xlim([-L // 10, L // 10])
axes[3].stem(freqs, X_magnitude, basefmt=" ", linefmt="m", markerfmt="mo")
axes[3].set_title("FFT - Magnitude  [zoomed in]")
axes[3].set_xlabel("Frequency Index")
axes[3].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[3].set_xlim([-L // 10, L // 10])
plt.show()

## <a id='toc2_2_'></a>[Cosine Transform](#toc0_)

- The DCT decomposes a signal into **only cosine components**, making it more **energy-efficient** for many signals.

**Properties:**
- **Output**: Only real values (no imaginary part).
- **Symmetry**: The DCT assumes the signal is evenly extended, leading to better energy compaction.
- **Energy compaction**: The low-frequency coefficients hold most of the signal energy.
- **Fast computation**: The Fast Cosine Transform (FCT) accelerates the DCT (like FFT for DFT).
- **Use cases**: Compression (JPEG, MPEG, MP3), feature extraction in machine learning, and numerical methods.

**Forward Discrete Cosine Transform:**
$$X_k = \sum_{n=0}^{N-1} x_n \cos \left( \frac{\pi}{N} \left( n + \frac{1}{2} \right) k \right), \quad k = 0, 1, 2, \dots, N-1$$

**Inverse Discrete Cosine Transform:**
$$x_n = \frac{2}{N} \sum_{k=0}^{N-1} X_k \cos \left( \frac{\pi}{N} \left( n + \frac{1}{2} \right) k \right), \quad n = 0, 1, 2, \dots, N-1$$


### <a id='toc2_2_1_'></a>[Example](#toc0_)

- The DCT will concentrate more of the energy in the first few coefficients, which is why it is commonly used in compression (e.g., JPEG or MP3).


In [ ]:
# parameters
N = 128  # length of the signal
t = np.arange(N)  # time vector
f1, f2 = 5, 20  # frequencies for sine wave components

# create a simple signal (sum of two sine waves)
signal = np.sin(2 * np.pi * f1 * t / N) + 0.5 * np.sin(2 * np.pi * f2 * t / N)

# compute the DFT of the signal
X_dft = sp.fft.fft(signal)

# compute the DCT of the signal (Type-II, most commonly used)
X_dct = sp.fft.dct(signal, type=2)

# energy compaction: calculate the cumulative energy in both transforms
energy_dft = np.cumsum(np.abs(X_dft) ** 2) / np.sum(np.abs(X_dft) ** 2)
energy_dct = np.cumsum(np.abs(X_dct) ** 2) / np.sum(np.abs(X_dct) ** 2)

# plot
fig, axes = plt.subplots(1, 4, figsize=(18, 4), layout="compressed")
axes[0].plot(t, signal, label="Original Signal", color="yellow")
axes[0].set_title("Original Signal")
axes[0].set_xlabel("Sample index")
axes[0].set_ylabel("Amplitude")
axes[0].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[1].plot(energy_dft, label="DFT Energy Compaction", color="red")
axes[1].plot(energy_dct, label="DCT Energy Compaction", color="green")
axes[1].set_title("Energy Compaction Comparison")
axes[1].set_xlabel("Number of Coefficients")
axes[1].set_ylabel("Cumulative Energy")
axes[1].legend()
axes[1].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[2].stem(np.abs(X_dft), basefmt=" ", linefmt="r-", markerfmt="ro", label="DFT Coefficients")
axes[2].set_title("DFT Coefficients")
axes[2].set_xlabel("Frequency Bin")
axes[2].set_ylabel("Magnitude")
axes[2].grid(True, linewidth=0.5, linestyle="--", color="gray")
axes[3].stem(np.abs(X_dct), basefmt=" ", linefmt="g-", markerfmt="go", label="DCT Coefficients")
axes[3].set_title("DCT Coefficients")
axes[3].set_xlabel("Coefficient Index")
axes[3].set_ylabel("Magnitude")
axes[3].grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()